In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

In [2]:
import os, sys, time

In [3]:
#import json
#aFile = json.loads("/afs/cern.ch/user/n/nmangane/LTW/FourTopNAOD/test/testjson.json")
#aFile = json.loads("{"1":[[145013,145100],[145178,145300]]}")
#aFile = json.load(open("testjson.json", 'r'))
#for run, lumi in aFile.iteritems():
#    print(str(run) + " -> " + str(lumi))

In [4]:
import ROOT

Welcome to JupyROOT 6.10/09


In [5]:
#Whoops, had to do scram b in src directory to build all the libraries and such... can't find the postproc otherwise.
#reload kernel as well if above done after failing to import postproc
from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

In [6]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

In [18]:
class testing(Module):
    def __init__(self):
        self.counting = 0
        pass
    def beginJob(self):
        pass
    def endJob(self):
        pass
    #def beginFile(self, inputFile, outputFile):
    #def beginFile(self):
        #pass
    #def endFile(self, inputFile, outputFile):
    #def endFile(self):
        #pass
    def analyze(self, event):
        """process event, return True (go to next module) or False (fail, go to next event)"""
        self.counting += 1
        if self.counting > 7:
            return False
        electrons = Collection(event, "Electron")
        photons = Collection(event, "Photon")
        muons = Collection(event, "Muon")
        taus = Collection(event, "Tau")
        jets = Collection(event, "Jet")
        met = Object(event, "MET")
        run = getattr(event, "run") #run = event.run works equally well, but this method is more powerful
        evt = getattr(event, "event")
        HLT = Object(event, "HLT") 
        Filters = Object(event, "Flag") 
        PV = Object(event, "PV")
        SV = Collection(event, "SV")
        genParts = Collection(event, "GenPart")
        genJets = Collection(event, "GenJet")
        genFatJets = Collection(event, "GenJetAK8")
        eventSum = ROOT.TLorentzVector()
        print("Run: {0:>8d} Event: {1:>8d}".format(run, evt))
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f} nDoF: {3: >5f} Chi^2: {4: >5.3f}".format(
            PV.x,PV.y, PV.z, PV.ndof, PV.chi2))
        if len(SV) > 0:   
            print("nSV: {0: >3d} SV[0] Decay Length:{1: >5.3f}".format(len(SV), SV[0].dlen ))
        else:
            print("nSV: {0: >3d}".format(len(SV)))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Muon", "Pt", "Eta", "Phi"))
        for nm, lep in enumerate(muons) :
            eventSum += lep.p4()
            #format_spec ::=  [[fill]align][sign][#][0][width][,][.precision][type]
            print("{0:*<5d} {1:>10.4f} {2:>+10.3f} {3:>+10.3f}".format(nm+1, lep.pt, lep.eta, lep.phi))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Electron", "Pt", "Eta", "Phi"))
        for ne, lep in enumerate(electrons) :
            eventSum += lep.p4()
            print("{0:*^5d} {1:>10.4f} {2:> 10.3f} {3:> 10.3f}".format(ne+1, lep.pt, lep.eta, lep.phi))
        #for j in filter(self.jetSel,jets):
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Jet", "Pt", "Eta", "Phi"))
        for nj, j in enumerate(jets):
            eventSum += j.p4()
            print("{0: >5d} {1:>10.4f} {2:>-10.3f} {3:>-10.3f}".format(nj+1, j.pt, j.eta, j.phi))
        #for nt, trig in enumerate(triggers):
        #    if(nt < 5): print("TypeName: " + trig.GetTypeName())
        #idea: create list of names for triggers, then check bits with "triggers.name for name in names"
        #Use getattr(triggers, variablename) to access!
        passTrig=["PFMETNoMu90_PFMHTNoMu90_IDTight"]
        for trig in passTrig:
            print("HLT_" + str(trig) + " Trigger: " + str(getattr(HLT, trig)) )
        passFilter=["HBHENoiseFilter", "HBHENoiseIsoFilter", "EcalDeadCellTriggerPrimitiveFilter", 
                    "globalSuperTightHalo2016Filter", "goodVertices", "METFilters"]
        for fltr in passFilter:
            print("Flag_" + str(fltr) + " Filter: " + str(getattr(Filters, fltr)))
        print("Event Mass: {:<10.4f}\n".format(eventSum.M()))
        return True

# define modules using the syntax 'name = lambda : constructor' to avoid having them loaded when not needed

# testing = lambda : testing(jetSelection= lambda j : j.pt > 30)

In [33]:
#preselection="Jet_pt[0] > 250"
preselection="nMuon > 0 && nElectron > 0"
files=[" root://cms-xrd-global.cern.ch//store/mc/RunIISummer16NanoAOD/TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/NANOAODSIM/PUMoriond17_\
05Feb2018_94X_mcRun2_asymptotic_v2-v1/40000/2CE738F9-C212-E811-BD0E-EC0D9A8222CE.root"]
print(files)

[u' root://cms-xrd-global.cern.ch//store/mc/RunIISummer16NanoAOD/TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/NANOAODSIM/PUMoriond17_05Feb2018_94X_mcRun2_asymptotic_v2-v1/40000/2CE738F9-C212-E811-BD0E-EC0D9A8222CE.root']


In [34]:
#p=PostProcessor(".",files,cut=preselection,branchsel=None,modules=[testing()],noOut=True,histFileName="histOut.root",histDirName="plots")
p=PostProcessor(".",files,cut=preselection,branchsel=None,modules=[testing()],friend=False,postfix="AccessAndLoop", jsonInput=None,
                noOut=False,justcount=False,provenance=False,haddFileName=None,fwkJobReport=False,histFileName=None,
                histDirName=None, outputbranchsel=None)

In [36]:
p.run()

Will write selected trees to .
Pre-select 468497 entries out of 1386834 
Run:        1 Event:   145013
PV  X: 0.106 Y: 0.170 Z: -3.108 nDoF: 95.500000 Chi^2: 0.879
nSV:   1 SV[0] Decay Length:0.599
 Muon         Pt        Eta        Phi
1****    10.4301     +2.212     +1.385
2****     5.7181     +0.355     -0.389
Electron         Pt        Eta        Phi
**1**     5.0823      0.550      1.295
  Jet         Pt        Eta        Phi
    1    59.5000      2.309      1.351
    2    54.5312      0.693     -2.048
    3    44.0000      0.252     -0.333
    4    40.8125      2.755     -0.990
    5    38.9688      1.495      1.870
    6    37.8125      1.443      2.516
    7    35.1562      1.525      2.925
    8    22.4688     -3.195      1.143
    9    17.8438      4.054     -0.340
HLT_PFMETNoMu90_PFMHTNoMu90_IDTight Trigger: False
Flag_HBHENoiseFilter Filter: True
Flag_HBHENoiseIsoFilter Filter: True
Flag_EcalDeadCellTriggerPrimitiveFilter Filter: True
Flag_globalSuperTightHalo2016Filter Fil